# ConvLSTM2D Experiment

## import

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
# check GPU availability
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices("GPU")))

In [ ]:
gpu_info = !nvidia-smi
gpu_info = "\n".join(gpu_info)
if gpu_info.find("failed") >= 0:
    print(
        'Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, '
    )
    print("and then re-execute this cell.")
else:
    print(gpu_info)

In [ ]:
import os

import numpy as np
import pandas as pd

# Setting seed for reproducibility
tf.random.set_seed(1234)
np.random.seed(1234)

# setup training log for tensorborad
workingfolder="."
# workingfolder="/content/drive/My Drive/Colab Notebooks"

log_base = os.path.join(workingfolder, "logs", "fit")

import io
import itertools
import shutil
import tempfile
import time

import sklearn.metrics

In [ ]:
from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()

import matplotlib as mpl
from matplotlib import dates as mdates
from matplotlib import pyplot as plt
from matplotlib import rc
from matplotlib.ticker import FormatStrFormatter

rc("font", **{"family": "sans-serif", "sans-serif": ["Helvetica"]})

import seaborn as sns

params = {
    "axes.labelsize": 8,
    "font.size": 6,
    "font.family": "sans-serif",
    "font.sans-serif": "Helvetica",
    "legend.fontsize": 8,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "text.usetex": False,
    "figure.figsize": [4, 4],
    #     'path.simplify': True,
    #     'xtick.major.size':   6,
    #     'ytick.major.size':   6,
    #     'xtick.minor.size':   3,
    #     'ytick.minor.size':   3,
    #     'xtick.major.width'       :   1,
    #     'ytick.major.width'       :   1,
    #     'xtick.minor.width'       :   1,
    #     'ytick.minor.width'       :   1,
    #     'lines.markeredgewidth'   :   1,
    "legend.frameon": False,
    #     'legend.handletextpad'    :   0.3,
}
mpl.rcParams.update(params)

colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

sns.set_context("paper")
sns.set(font="Helvetica")
sns.set_style("white", {"font.family": "sans-serif", "font.serif": ["Helvetica"]})

import datetime

from matplotlib import units as munits

formats = [
    "%y",  # ticks are mostly years
    "%b",  # ticks are mostly months
    "%d",  # ticks are mostly days
    "%H:%M",  # hrs
    "%H:%M",  # min
    "%S.%f",
]  # secs
# these can be the same, except offset by one level....
zero_formats = [""] + formats[:-1]
# ...except for ticks that are mostly hours, then its nice to have month-day
zero_formats[3] = "%d-%b"
offset_formats = [
    "",
    "%Y",
    "%b %Y",
    "%d %b %Y",
    "%d %b %Y",
    "%d %b %Y %H:%M",
]

converter = mdates.ConciseDateConverter(
    formats=formats, zero_formats=zero_formats, offset_formats=offset_formats
)

munits.registry[np.datetime64] = converter
munits.registry[datetime.date] = converter
munits.registry[datetime.datetime] = converter

In [ ]:
# os.environ["PATH"] += os.pathsep + 'C:\Program Files (x86)\Graphviz2.38/bin/'

In [ ]:
def createf(path=None):
    try:
        os.makedirs(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)

In [ ]:
createf("./figs")
createf("./models")
createf("./run")
createf("./logs/fit")

In [ ]:
try:
    shutil.rmtree(log_base)
except OSError as e:
    print("Error: %s : %s" % (log_base, e.strerror))

In [ ]:
# Load the TensorBoard notebook extension, (it is not working for jupyternote book in vscode)
# for vscode, open terminal and run: (in this folder)
# tensorboard --logdir=script\logs
# %load_ext tensorboard
# %tensorboard --logdir logs


In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

## Data Ingestion

In [ ]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time index")
    plt.ylabel("Monthly Mean Total Sunspot Number")
    # plt.grid(True)

In [ ]:
import csv
time_step = []
sunspots = []

filepath="../input/sunspots/Sunspots.csv"

with open(filepath) as csvfile:
  reader = csv.reader(csvfile, delimiter=',')
  next(reader)
  for row in reader:
    sunspots.append(float(row[2]))
    time_step.append(int(row[0]))

series = np.array(sunspots)
time = np.array(time_step)
fig = plt.figure(figsize=(6, 4))
plot_series(time, series)

fig_name=os.path.join(workingfolder, 'figs','exp_algsel'+'_1_'+'sunspots'+'.pdf')
fig.savefig(fig_name,dpi=600, bbox_inches="tight")

In [ ]:
split_time = 2800
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

In [ ]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i)
    # Reshape data from (history_size,) to (history_size, 1)
    data.append(np.reshape(dataset[indices], (history_size, 1)))
    labels.append(dataset[i+target_size])
  return np.array(data), np.array(labels)

In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [ ]:
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [ ]:
# tf.keras.backend.clear_session()
# tf.keras.random.set_seed(51)
np.random.seed(51)
past_history = 60
future_target = 30
STEP = 1
EPOCH=1000
BATCH_SIZE=128

xs_train, y_train = univariate_data(x_train, 0, None, past_history, future_target)
xs_test,y_test =univariate_data(x_valid, 0, None, past_history, future_target)

In [ ]:
y_train = np.expand_dims(y_train, axis=1)
y_test = np.expand_dims(y_test, axis=1)

In [ ]:
Max = xs_train.max()
Min = xs_train.min()
print(Max,Min)

In [ ]:
xs_train_norm = (xs_train - Min)/(Max - Min)

xs_test_norm = (xs_test - Min)/(Max - Min)

## Model Build

### CNN + LSTM stack

In [ ]:
def model_build():
    tf.keras.backend.clear_session()
    model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=5,
              strides=1, padding="causal",
              activation="relu",
              input_shape=[None, 1]),
    tf.keras.layers.LSTM(32), # return_sequences=True
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * (Max-Min) + Min)
    ])
    model.compile(loss='mae',
              optimizer=tf.keras.optimizers.RMSprop(),
              metrics=["mape"])
    return model    

In [ ]:
model_CNNLSTM=model_build()
model_CNNLSTM.summary()

In [ ]:
model_name='model_CNNLSTM_univariate_singlestep_IDs'

model_dir=os.path.join(workingfolder,'models')

log_dir = os.path.join(log_base, datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+'_'+model_name)

# define path to save model
model_path =os.path.join(model_dir,model_name+'.h5') 

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, verbose=1, mode='auto')
checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='auto', verbose=0)
csv_logger = tf.keras.callbacks.CSVLogger(os.path.join(workingfolder, "run",model_name+"_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_history_log.csv"), append=True)

CALLBACKS = [tensorboard_callback, earlystopping, checkpoint, csv_logger]

In [ ]:
fig_name=os.path.join(workingfolder,'figs',model_name+'_modelplot'+'.png')

tf.keras.utils.plot_model(model_CNNLSTM, fig_name, show_shapes=True, show_layer_names=False, rankdir='LR', dpi=300)

In [ ]:
history_CNNLSTM = model_CNNLSTM.fit(xs_train_norm, y_train, epochs=EPOCH, batch_size=BATCH_SIZE, validation_split = 0.2,
                    callbacks=CALLBACKS)

In [ ]:
scores_test_CNNLSTM = model_CNNLSTM.evaluate(xs_test_norm,y_test, verbose=2)

In [ ]:
y_pred_test=model_CNNLSTM.predict(xs_test_norm)
y_true_test=y_test

In [ ]:
y_pred_test.shape

In [ ]:
y_true_test.shape

In [ ]:
fig = plt.figure(figsize=(6, 3))
plt.plot(y_pred_test.flatten(), color="red")
plt.plot(y_true_test.flatten(), color="green")
# plt.title('prediction')
plt.ylabel('Flow')
plt.xlabel('row')
plt.legend(['Predicted', 'True'], loc='best')
plt.show()

### CNN stack

In [ ]:
def model_build():
    tf.keras.backend.clear_session()
    model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=5,
              strides=1, padding="causal",
              activation="relu",
              input_shape=[past_history, 1]),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * (Max-Min) + Min)
    ])
    model.compile(loss='mae',
              optimizer=tf.keras.optimizers.RMSprop(),
              metrics=["mape"])
    return model   

In [ ]:
model_CNN=model_build()
model_CNN.summary()

In [ ]:
model_name='model_CNN_univariate_singlestep_IDs'

model_dir=os.path.join(workingfolder,'models')

log_dir = os.path.join(log_base, datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+'_'+model_name)

# define path to save model
model_path =os.path.join(model_dir,model_name+'.h5') 

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, verbose=1, mode='auto')
checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='auto', verbose=0)
csv_logger = tf.keras.callbacks.CSVLogger(os.path.join(workingfolder, "run",model_name+"_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_history_log.csv"), append=True)

CALLBACKS = [tensorboard_callback, earlystopping, checkpoint, csv_logger]

In [ ]:
fig_name=os.path.join(workingfolder,'figs',model_name+'_modelplot'+'.png')

tf.keras.utils.plot_model(model_CNN, fig_name, show_shapes=True, show_layer_names=False, rankdir='LR', dpi=300)

In [ ]:
history_CNN = model_CNN.fit(xs_train_norm, y_train, epochs=EPOCH, batch_size=BATCH_SIZE, validation_split = 0.2,
                    callbacks=CALLBACKS)

In [ ]:
scores_test_CNN = model_CNN.evaluate(xs_test_norm,y_test, verbose=2)

In [ ]:
y_pred_test=model_CNN.predict(xs_test_norm)
y_true_test=y_test

In [ ]:
fig = plt.figure(figsize=(6, 3))
plt.plot(y_pred_test.flatten(), color="red")
plt.plot(y_true_test.flatten(), color="green")
# plt.title('prediction')
plt.ylabel('Flow')
plt.xlabel('row')
plt.legend(['Predicted', 'True'], loc='best')
plt.show()

### LSTM stack

In [ ]:
def model_build2():
    model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True,input_shape=[past_history, 1]),
    tf.keras.layers.LSTM(32), # , return_sequences=True
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * (Max-Min) + Min)
    ])
    model.compile(loss='mae',
              optimizer=tf.keras.optimizers.RMSprop(),
              metrics=["mape"])
    return model   

In [ ]:
model_LSTM=model_build2()
model_LSTM.summary()

In [ ]:
model_name='model_LSTM_univariate_singlestep_IDs'
model_dir=os.path.join(workingfolder,'models')

log_dir = os.path.join(log_base, datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+'_'+model_name)

# define path to save model
model_path =os.path.join(model_dir,model_name+'.h5') 

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, verbose=1, mode='auto')
checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='auto', verbose=0)
csv_logger = tf.keras.callbacks.CSVLogger(os.path.join(workingfolder, "run",model_name+"_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_history_log.csv"), append=True)

CALLBACKS = [tensorboard_callback, earlystopping, checkpoint, csv_logger]

In [ ]:
fig_name=os.path.join(workingfolder,'figs',model_name+'_modelplot'+'.png')

tf.keras.utils.plot_model(model_LSTM, fig_name, show_shapes=True, show_layer_names=False,
    rankdir='LR', expand_nested=True, dpi=300)

In [ ]:

history_LSTM = model_LSTM.fit(xs_train_norm, y_train, epochs=EPOCH, batch_size=BATCH_SIZE, validation_split = 0.2,
                    callbacks=CALLBACKS)


In [ ]:
# rnn_forecast = model_forecast(model1, series[..., np.newaxis], window_size)
# rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

In [ ]:
# plt.figure(figsize=(10, 6))
# plot_series(time_valid, x_valid)
# plot_series(time_valid, rnn_forecast)

In [ ]:
scores_test_LSTM = model_LSTM.evaluate(xs_test_norm,y_test, verbose=2)

In [ ]:
y_pred_test=model_LSTM.predict(xs_test_norm)
y_true_test=y_test

In [ ]:
fig = plt.figure(figsize=(6, 3))
plt.plot(y_pred_test.flatten(), color="red")
plt.plot(y_true_test.flatten(), color="green")
# plt.title('prediction')
plt.ylabel('Flow')
plt.xlabel('row')
plt.legend(['Predicted', 'True'], loc='best')
plt.show()

### BiLSTM stack

In [ ]:
def model_build_biLSTM():
    model = tf.keras.models.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True), input_shape=[past_history, 1]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)), # , return_sequences=True
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * (Max-Min) + Min)
    ])
    model.compile(loss='mae',
              optimizer=tf.keras.optimizers.RMSprop(),
              metrics=["mape"])
    return model   

In [ ]:
model_BiLSTM=model_build_biLSTM()
model_BiLSTM.summary()

In [ ]:
model_name='model_BiLSTM_univariate_singlestep_IDs'
model_dir=os.path.join(workingfolder,'models')

log_dir = os.path.join(log_base, datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+'_'+model_name)

# define path to save model
model_path =os.path.join(model_dir,model_name+'.h5') 

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, verbose=1, mode='auto')
checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='auto', verbose=0)
csv_logger = tf.keras.callbacks.CSVLogger(os.path.join(workingfolder, "run",model_name+"_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_history_log.csv"), append=True)

CALLBACKS = [tensorboard_callback, earlystopping, checkpoint, csv_logger]

In [ ]:
fig_name=os.path.join(workingfolder,'figs',model_name+'_modelplot'+'.png')

tf.keras.utils.plot_model(model_BiLSTM, fig_name, show_shapes=True, show_layer_names=False,
    rankdir='LR', expand_nested=True, dpi=300)

In [ ]:
history_BiLSTM = model_BiLSTM.fit(xs_train_norm, y_train, epochs=EPOCH, batch_size=BATCH_SIZE, validation_split = 0.2,
                    callbacks=CALLBACKS)

In [ ]:
scores_test_BiLSTM = model_BiLSTM.evaluate(xs_test_norm,y_test, verbose=2)

In [ ]:
y_pred_test=model_BiLSTM.predict(xs_test_norm)
y_true_test=y_test

In [ ]:
fig = plt.figure(figsize=(6, 3))
plt.plot(y_pred_test.flatten(), color="red")
plt.plot(y_true_test.flatten(), color="green")
# plt.title('prediction')
plt.ylabel('Flow')
plt.xlabel('row')
plt.legend(['Predicted', 'True'], loc='best')
plt.show()

### ConvLSTM2D stack

In [ ]:
y_train=y_train.reshape((y_train.shape[0],1))

In [ ]:
n_timesteps, n_features, n_outputs = xs_train_norm.shape[1], xs_train_norm.shape[2], y_train.shape[1]
print(n_timesteps, n_features, n_outputs)

In [ ]:
# reshape into subsequences [samples, time steps, rows, cols, channels]
n_steps = 2
n_length = int (xs_train_norm.shape[1] /n_steps)
n_features = xs_train_norm.shape[-1]

print(n_length, n_features)

In [ ]:
x_5d= xs_train_norm.reshape((xs_train_norm.shape[0], n_steps, 1, n_length, n_features))
x_5d.shape

In [ ]:
train_y = y_train.reshape((y_train.shape[0], y_train.shape[1], 1))

In [ ]:
train_y.shape

In [ ]:
def model_build3():
    model = tf.keras.models.Sequential([
    tf.keras.layers.ConvLSTM2D(filters=64, kernel_size=(1,3), 
                               activation='relu',
#                                padding='same',
                               input_shape=(n_steps, 1, n_length, n_features), return_sequences=True),
#     tf.keras.layers.ConvLSTM2D(filters=32, kernel_size=(1,3),
#                                padding='same',
#                                activation='relu',return_sequences=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.RepeatVector(n_outputs),
    tf.keras.layers.LSTM(64, activation='relu', return_sequences=True),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(30, activation='relu')),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1)),
    tf.keras.layers.Lambda(lambda x: x * (Max-Min) + Min)
    ])
    model.compile(loss='mae',
              optimizer=tf.keras.optimizers.RMSprop(),
              metrics=["mape"])
    return model   

In [ ]:
model_ConvLSTM2D=model_build3()
model_ConvLSTM2D.summary()

In [ ]:
model_name='model_CONVLSTM2D_univariate_singlestep_IDs'
model_dir=os.path.join(workingfolder,'models')

log_dir = os.path.join(log_base, datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+'_'+model_name)

# define path to save model
model_path =os.path.join(model_dir,model_name+'.h5') 

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, verbose=1, mode='auto')
checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='auto', verbose=0)
csv_logger = tf.keras.callbacks.CSVLogger(os.path.join(workingfolder, "run",model_name+"_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_history_log.csv"), append=True)

CALLBACKS = [tensorboard_callback, earlystopping, checkpoint, csv_logger]

In [ ]:
fig_name=os.path.join(workingfolder,'figs',model_name+'_modelplot'+'.png')

tf.keras.utils.plot_model(model_ConvLSTM2D, fig_name, show_shapes=True, show_layer_names=False,
    rankdir='LR', expand_nested=True, dpi=300)

In [ ]:
history_ConvLSTM2D = model_ConvLSTM2D.fit(x_5d,train_y,  epochs=EPOCH, batch_size=BATCH_SIZE, validation_split = 0.2,
                    callbacks=CALLBACKS)


In [ ]:
xs_test_norm.shape

In [ ]:
x_val_5d=xs_test_norm.reshape(xs_test_norm.shape[0], n_steps, 1, n_length, n_features)
y_val_reshaped=y_test.reshape((y_test.shape[0], 1, 1))

In [ ]:
y_pred_test=model_ConvLSTM2D.predict(x_val_5d)
y_true_test=y_val_reshaped

In [ ]:
scores_test_ConvLSTM2D = model_ConvLSTM2D.evaluate(x_val_5d, y_val_reshaped, verbose=2)

In [ ]:
y_true_test.shape

In [ ]:
y_pred_test.shape

In [ ]:
fig = plt.figure(figsize=(6, 3))
plt.plot(y_pred_test.flatten(), color="red")
plt.plot(y_true_test.flatten(), color="green")
# plt.title('prediction')
plt.ylabel('Flow')
plt.xlabel('row')
plt.legend(['Predicted', 'True'], loc='best')
plt.show()

### simple RNN stack

In [ ]:
def model_build4():
    model = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(32, return_sequences=True,input_shape=[past_history, 1]),
    tf.keras.layers.SimpleRNN(64), # , return_sequences=True
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * (Max-Min) + Min)
    ])
    model.compile(loss='mae',
              optimizer=tf.keras.optimizers.RMSprop(),
              metrics=["mape"])
    return model

model_RNN=model_build4()
model_RNN.summary()

In [ ]:
model_name='model_RNN_univariate_singlestep_IDs'
model_dir=os.path.join(workingfolder,'models')

log_dir = os.path.join(log_base, datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+'_'+model_name)

# define path to save model
model_path =os.path.join(model_dir,model_name+'.h5') 

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, verbose=1, mode='auto')
checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='auto', verbose=0)
csv_logger = tf.keras.callbacks.CSVLogger(os.path.join(workingfolder, "run",model_name+"_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_history_log.csv"), append=True)

CALLBACKS = [tensorboard_callback, earlystopping, checkpoint, csv_logger]

In [ ]:
fig_name=os.path.join(workingfolder,'figs',model_name+'_modelplot'+'.png')

tf.keras.utils.plot_model(model_RNN, fig_name, show_shapes=True, show_layer_names=False,
    rankdir='LR', expand_nested=True, dpi=300)

In [ ]:
history_RNN = model_RNN.fit(xs_train_norm, y_train, epochs=EPOCH, batch_size=BATCH_SIZE, validation_split = 0.2,
                    callbacks=CALLBACKS)

In [ ]:
scores_test_RNN = model_RNN.evaluate(xs_test_norm,y_test, verbose=2)

In [ ]:
fig = plt.figure(figsize=(6, 3))
plt.plot(y_pred_test.flatten(), color="red")
plt.plot(y_true_test.flatten(), color="green")
# plt.title('prediction')
plt.ylabel('Value')
plt.xlabel('row')
plt.legend(['Predicted', 'True'], loc='best')
plt.show()

### GRU stack

In [ ]:
def model_build5():
    model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(32, return_sequences=True,input_shape=[past_history, 1]),
    tf.keras.layers.GRU(64), # , return_sequences=True
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * (Max-Min) + Min)
    ])
    model.compile(loss='mae',
              optimizer=tf.keras.optimizers.RMSprop(),
              metrics=["mape"])
    return model

model_GUR=model_build5()
model_GUR.summary()

In [ ]:
model_name='model_GUR_univariate_singlestep_IDs'
model_dir=os.path.join(workingfolder,'models')

log_dir = os.path.join(log_base, datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+'_'+model_name)

# define path to save model
model_path =os.path.join(model_dir,model_name+'.h5') 

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100, verbose=1, mode='auto')
checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='auto', verbose=0)
csv_logger = tf.keras.callbacks.CSVLogger(os.path.join(workingfolder, "run",model_name+"_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+"_history_log.csv"), append=True)

CALLBACKS = [tensorboard_callback, earlystopping, checkpoint, csv_logger]

In [ ]:
fig_name=os.path.join(workingfolder,'figs',model_name+'_modelplot'+'.png')

tf.keras.utils.plot_model(model_GUR, fig_name, show_shapes=True, show_layer_names=False,
    rankdir='LR', expand_nested=True, dpi=300)

In [ ]:
history_GUR = model_GUR.fit(xs_train_norm, y_train, epochs=EPOCH, batch_size=BATCH_SIZE, validation_split = 0.2,
                    callbacks=CALLBACKS)


In [ ]:
scores_test_GUR = model_GUR.evaluate(xs_test_norm,y_test, verbose=2)

In [ ]:
fig = plt.figure(figsize=(6, 3))
plt.plot(y_pred_test.flatten(), color="red")
plt.plot(y_true_test.flatten(), color="green")
# plt.title('prediction')
plt.ylabel('Value')
plt.xlabel('Row')
plt.legend(['Predicted', 'True'], loc='best')
plt.show()

## Summary

In [ ]:
y_pred_test_RNN=model_RNN.predict(xs_test_norm)
y_pred_test_GUR=model_GUR.predict(xs_test_norm)
y_pred_test_LSTM=model_LSTM.predict(xs_test_norm)
y_pred_test_BiLSTM=model_BiLSTM.predict(xs_test_norm)
y_pred_test_CNNLSTM=model_CNNLSTM.predict(xs_test_norm)
y_pred_test_CNN=model_CNN.predict(xs_test_norm)
y_pred_test_ConvLSTM2D =model_ConvLSTM2D.predict(x_val_5d)

In [ ]:
y_pred_train_RNN=model_RNN.predict(xs_train_norm)
y_pred_train_GUR=model_GUR.predict(xs_train_norm)
y_pred_train_LSTM=model_LSTM.predict(xs_train_norm)
y_pred_train_BiLSTM=model_BiLSTM.predict(xs_train_norm)
y_pred_train_CNNLSTM=model_CNNLSTM.predict(xs_train_norm)
y_pred_train_CNN=model_CNN.predict(xs_train_norm)
y_pred_train_ConvLSTM2D =model_ConvLSTM2D.predict(x_5d)

In [ ]:
mae_test_baseline= np.mean(np.abs(y_true_test[future_target+1:].flatten()-y_true_test[:-1-future_target].flatten()))
mae_test_RNN= np.mean(np.abs(y_pred_test_RNN.flatten()-y_true_test.flatten()))
mae_test_GUR= np.mean(np.abs(y_pred_test_GUR.flatten()-y_true_test.flatten()))
mae_test_LSTM= np.mean(np.abs(y_pred_test_LSTM.flatten()-y_true_test.flatten()))
mae_test_BiLSTM= np.mean(np.abs(y_pred_test_BiLSTM.flatten()-y_true_test.flatten()))
mae_test_CNNLSTM= np.mean(np.abs(y_pred_test_CNNLSTM.flatten()-y_true_test.flatten()))
mae_test_CNN= np.mean(np.abs(y_pred_test_CNN.flatten()-y_true_test.flatten()))
mae_test_ConvLSTM2D= np.mean(np.abs(y_pred_test_ConvLSTM2D.flatten()-y_true_test.flatten()))

In [ ]:
mae_train_baseline= np.mean(np.abs(y_train[future_target+1::].flatten()-y_train[:-1-future_target].flatten()))
mae_train_RNN= np.mean(np.abs(y_pred_train_RNN.flatten()-y_train.flatten()))
mae_train_GUR= np.mean(np.abs(y_pred_train_GUR.flatten()-y_train.flatten()))
mae_train_LSTM= np.mean(np.abs(y_pred_train_LSTM.flatten()-y_train.flatten()))
mae_train_BiLSTM= np.mean(np.abs(y_pred_train_BiLSTM.flatten()-y_train.flatten()))
mae_train_CNNLSTM= np.mean(np.abs(y_pred_train_CNNLSTM.flatten()-y_train.flatten()))
mae_train_CNN= np.mean(np.abs(y_pred_train_CNN.flatten()-y_train.flatten()))
mae_train_ConvLSTM2D= np.mean(np.abs(y_pred_train_ConvLSTM2D.flatten()-train_y.flatten()))

In [ ]:
mae_test_RNN

In [ ]:
# 
# plt.style.use('seaborn-paper')
# sns.set_context("paper")

labels = ['Baseline', 'CNN', 'RNN', 'GUR', 'LSTM', 'BiLSTM', 'CNNLSTM','ConvLSTM']
test_mae = [mae_test_baseline, mae_test_CNN, mae_test_RNN, mae_test_GUR,mae_test_LSTM,mae_test_BiLSTM,mae_test_CNNLSTM,mae_test_ConvLSTM2D]
# training_mae = [np.mean(history_RNN.history['loss']),
#                     np.mean(history_GUR.history['loss']),
#                     np.mean(history_LSTM.history['loss']),
#                     np.mean(history_CNNLSTM.history['loss']),
#                     np.mean(history_ConvLSTM2D.history['loss'])]

# val_mae = [np.min(history_RNN.history['val_loss']),
#                 np.mean(history_GUR.history['val_loss']),
#                 np.mean(history_LSTM.history['val_loss']),
#                 np.mean(history_CNNLSTM.history['val_loss']),
#                 np.mean(history_ConvLSTM2D.history['val_loss'])]

training_mae = [mae_train_baseline,mae_train_CNN, mae_train_RNN,mae_train_GUR,mae_train_LSTM,mae_train_BiLSTM, mae_train_CNNLSTM,mae_train_ConvLSTM2D]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(7, 4))
rects1 = ax.bar(x - width/2, training_mae, width, label='train')
# rects2 = ax.bar(x , val_mae, width, label='val')
rects3 = ax.bar(x + width/2, test_mae, width, label='test')

# rects4 = ax.plot([mae_test_baseline,mae_test_baseline,mae_test_baseline,mae_test_baseline,mae_test_baseline], label='baseline')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Loss-MAE')
# ax.set_title('Scores by group and gender')
ax.set_xticks(x)
ax.set_xticklabels(labels)

ax.legend(loc="best" ) #,bbox_to_anchor=(1,0.5)

fig.tight_layout()

plt.ylim(10, 90)

plt.show()

fig_name=os.path.join(workingfolder,'figs','exp_algsel'+'tainvaltest'+'.pdf')
fig.savefig(fig_name,dpi=600, bbox_inches="tight")

In [ ]:
fig = plt.figure(figsize=(14, 8))

# plt.style.use('seaborn-paper')
# sns.set_context("paper")

ax1 = plt.subplot(231)

plt.plot(y_pred_test_RNN, color="red",label='RNN')
plt.plot(y_true_test.flatten(), color="green", label='True')
plt.legend()

ax1 = plt.subplot(232)
plt.plot(y_pred_test_GUR, color="red",label='GUR')
plt.plot(y_true_test.flatten(), color="green", label='True')
plt.legend()

ax1 = plt.subplot(233)
plt.plot(y_pred_test_LSTM,color="red", label='LSTM')
plt.plot(y_true_test.flatten(), color="green", label='True')
plt.legend()

ax1 = plt.subplot(234)
plt.plot(y_pred_test_BiLSTM, color="red",label='LSTM')
plt.plot(y_true_test.flatten(), color="green", label='True')
plt.legend()


ax1 = plt.subplot(235)
plt.plot(y_pred_test_CNNLSTM,color="red", label='CNNLSTM')
plt.plot(y_true_test.flatten(), color="green", label='True')
plt.legend()

ax1 = plt.subplot(236)
# plt.plot(y_true_test[1:].flatten(), label='Baseline')
# plt.plot(y_true_test[:-1].flatten(), color="green")
# plt.legend()

plt.plot(y_pred_test_ConvLSTM2D.flatten(), color="red",label='ConvLSTM2D')
plt.plot(y_true_test.flatten(), color="green" ,label='True')
plt.legend()


fig_name=os.path.join(workingfolder,'figs','exp_algsel'+'testprediction'+'.pdf')
fig.savefig(fig_name,dpi=600, bbox_inches="tight")


# 

In [ ]:
fig = plt.figure(figsize=(10, 6))
# plt.style.use('seaborn-paper')
# sns.set_context("paper")

ax1 = plt.subplot(231)

test_mae_loss = np.abs(y_true_test.flatten() - y_pred_test_RNN.flatten())
sns.distplot(test_mae_loss, bins=30, kde=True)
plt.legend(['RNN'], loc='best')


# test_mae_loss = np.abs(y_true_test[1:].flatten() -y_true_test[:-1].flatten())
# sns.distplot(test_mae_loss, bins=30, kde=True)
# plt.legend(['Baseline'], loc='best')

ax2 = plt.subplot(232, sharex=ax1, sharey=ax1)

test_mae_loss = np.abs(y_true_test.flatten() -y_pred_test_GUR.flatten())
sns.distplot(test_mae_loss, bins=30, kde=True)
plt.legend(['GUR'], loc='best')


ax3 = plt.subplot(233, sharex=ax1, sharey=ax1)
test_mae_loss = np.abs(y_true_test.flatten() -y_pred_test_LSTM.flatten())
sns.distplot(test_mae_loss, bins=30, kde=True)
plt.legend(['LSTM'], loc='best')

ax4 = plt.subplot(234, sharex=ax1, sharey=ax1)
test_mae_loss = np.abs(y_true_test.flatten() -y_pred_test_BiLSTM.flatten())
sns.distplot(test_mae_loss, bins=30, kde=True)
plt.legend(['BiLSTM'], loc='best')

ax5 = plt.subplot(235, sharex=ax1, sharey=ax1)

test_mae_loss = np.abs(y_true_test.flatten() -y_pred_test_CNNLSTM.flatten())
sns.distplot(test_mae_loss, bins=30, kde=True)
plt.legend(['CNN+LSTM'], loc='best')



ax6 = plt.subplot(236, sharex=ax1, sharey=ax1)

test_mae_loss = np.abs(y_true_test.flatten() -y_pred_test_ConvLSTM2D.flatten())
sns.distplot(test_mae_loss, bins=30, kde=True)
plt.legend(['ConvLSTM'], loc='best')


fig.text(0.5, 0.04, 'mae', ha='center')
fig.text(0.04, 0.5, 'distribution', va='center', rotation='vertical')
# plt.legend()

fig_name=os.path.join(workingfolder,'figs','exp_algsel'+'_3_'+'training history2'+'.pdf')
fig.savefig(fig_name,dpi=600, bbox_inches="tight")

In [ ]:
# summarize history for Loss
fig = plt.figure(figsize=(8, 6))

plt.plot(history_RNN.history['loss'])
plt.plot(history_RNN.history['val_loss'])
plt.plot(history_CNN.history['loss'])
plt.plot(history_CNN.history['val_loss'])

plt.plot(history_GUR.history['loss'])
plt.plot(history_GUR.history['val_loss'])
plt.plot(history_LSTM.history['loss'])
plt.plot(history_LSTM.history['val_loss'])
plt.plot(history_BiLSTM.history['loss'])
plt.plot(history_BiLSTM.history['val_loss'])
plt.plot(history_CNNLSTM.history['loss'])
plt.plot(history_CNNLSTM.history['val_loss'])
plt.plot(history_ConvLSTM2D.history['loss'])
plt.plot(history_ConvLSTM2D.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='best')
# plt.ylim(18, 30)
# plt.xlim(50, 250)
plt.show()

In [ ]:
y_len=len(history_RNN.history['loss'])

# plt.style.use('seaborn-paper')
# sns.set_context("paper")

df = pd.DataFrame(dict(epoch=np.arange(y_len),
                       mae=history_RNN.history['loss'], Type='train', DL= 'RNN'))
df=df.append(pd.DataFrame(dict(epoch=np.arange(y_len),
                       mae=history_RNN.history['val_loss'], Type='val', DL= 'RNN')))

df = pd.DataFrame(dict(epoch=np.arange(len(history_CNN.history['loss'])),
                       mae=history_CNN.history['loss'], Type='train', DL= 'CNN'))
df=df.append(pd.DataFrame(dict(epoch=np.arange(len(history_CNN.history['loss'])),
                       mae=history_CNN.history['val_loss'], Type='val', DL= 'CNN')))

df=df.append(pd.DataFrame(dict(epoch=np.arange(len(history_LSTM.history['loss'])),
                       mae=history_LSTM.history['loss'], Type='train', DL= 'LSTM')))
df=df.append(pd.DataFrame(dict(epoch=np.arange(len(history_LSTM.history['loss'])),
                       mae=history_LSTM.history['val_loss'], Type='val', DL= 'LSTM')))

df=df.append(pd.DataFrame(dict(epoch=np.arange(len(history_BiLSTM.history['loss'])),
                       mae=history_BiLSTM.history['loss'], Type='train', DL= 'BiLSTM')))
df=df.append(pd.DataFrame(dict(epoch=np.arange(len(history_BiLSTM.history['loss'])),
                       mae=history_BiLSTM.history['val_loss'], Type='val', DL= 'BiLSTM')))

df=df.append(pd.DataFrame(dict(epoch=np.arange(len(history_GUR.history['loss'])),
                       mae=history_GUR.history['loss'], Type='train', DL= 'GUR')))
df=df.append(pd.DataFrame(dict(epoch=np.arange(len(history_GUR.history['loss'])),
                       mae=history_GUR.history['val_loss'], Type='val', DL= 'GUR')))

df=df.append(pd.DataFrame(dict(epoch=np.arange(len(history_CNNLSTM.history['loss'])),
                       mae=history_CNNLSTM.history['loss'], Type='train', DL= 'CNNLSTM')))
df=df.append(pd.DataFrame(dict(epoch=np.arange(len(history_CNNLSTM.history['loss'])),
                       mae=history_CNNLSTM.history['val_loss'], Type='val', DL= 'CNNLSTM')))

df=df.append(pd.DataFrame(dict(epoch=np.arange(len(history_ConvLSTM2D.history['loss'])),
                       mae=history_ConvLSTM2D.history['loss'], Type='train', DL= 'ConvLSTM')))
df=df.append(pd.DataFrame(dict(epoch=np.arange(len(history_ConvLSTM2D.history['loss'])),
                       mae=history_ConvLSTM2D.history['val_loss'], Type='val', DL= 'ConvLSTM')))

sns.relplot(x="epoch", y="mae",
            hue="DL",col='Type',height=5,aspect=1.25,legend='full',
            kind="line", data=df)
sns.set_context("paper")

# plt.ylim(15, 23)
# plt.xlim(50, 250)

# plt.ylim(18, 30)
# plt.xlim(50, 250)

fig_name=os.path.join(workingfolder,'figs','exp_algsel'+'_2_'+'training history'+'.pdf')
fig.savefig(fig_name,dpi=600, bbox_inches="tight")

In [ ]:
history_CNN

In [ ]:
fig = plt.figure(figsize=(10, 10))

# plt.style.use('seaborn-paper')
# sns.set_context("paper")

sns.lineplot(x="epoch", y="mae", 
            style="Type",hue='DL',
            data=df);

# plt.ylim(15, 23)
# plt.xlim(50, 250)

# plt.ylim(18, 40)
# plt.xlim(100, 250)


plt.legend(bbox_to_anchor=(1,0.7))

fig_name=os.path.join(workingfolder,'figs','exp_algsel'+'_3_'+'training history3'+'.pdf')
plt.savefig(fig_name,dpi=600, bbox_inches="tight")

In [ ]:
fig = plt.figure(figsize=(6, 4))

# plt.style.use('seaborn-paper')
# sns.set_context("paper")

sns.lineplot(x="epoch", y="mae", 
            style="Type",hue='DL',
            data=df);

# plt.ylim(15, 23)
# plt.xlim(50, 250)

plt.ylim(18, 40)
# plt.xlim(100, 250)


plt.legend(bbox_to_anchor=(1,0.7))

fig_name=os.path.join(workingfolder,'figs','exp_algsel'+'_3_'+'training history'+'.pdf')
plt.savefig(fig_name,dpi=600, bbox_inches="tight")

In [ ]:
# fig = plt.figure(figsize=(18, 10))

# # fig, (ax0, ax1, ax2) = plt.subplots(ncols=3, constrained_layout=True)
# # t = np.arange(0, y_true_test[0], 1)

# ax1 = plt.subplot(331)

# plt.plot(y_pred_test2[:,-1,0], color="red")
# plt.plot(y_true_test.flatten(), color="green")
# plt.title('LSTM')
# plt.ylabel('value')
# # # plt.xlabel('Row')
# plt.legend(['predicted', 'actual data'], loc='best')

# # plt.setp(ax1.get_xticklabels(), fontsize=6)

# ax2 = plt.subplot(332, sharex=ax1, sharey=ax1)

# plt.plot(y_pred_test1[:,-1,0], color="red")
# plt.plot(y_true_test.flatten(), color="green")
# plt.title('CNN+LSTM')
# # plt.ylabel('value')
# plt.xlabel('timesteps')
# plt.legend(['predicted', 'actual data'], loc='best')


# # plt.setp(ax2.get_xticklabels(), visible=False)

# ax3 = plt.subplot(333, sharex=ax1, sharey=ax1)

# plt.plot(y_pred_test3.flatten(), color="red")
# plt.plot(y_true_test.flatten(), color="green")
# plt.title('ConvLTSM2D')
# # plt.ylabel('Value')
# # plt.xlabel('Row')
# plt.legend(['predicted', 'actual data'], loc='best')


# ax4 = plt.subplot(334 )

# plt.plot(history2.history['mae'])
# plt.plot(history2.history['val_mae'])
# # # plt.title('model loss')
# plt.ylabel('mae (training)')
# # plt.xlabel('training epoch')
# plt.legend(['train', 'val'], loc='best')



# ax5 = plt.subplot(335, sharex=ax4,sharey=ax4)

# plt.plot(history1.history['mae'])
# plt.plot(history1.history['val_mae'])
# # plt.title('model loss')
# # plt.ylabel('mae')
# plt.xlabel('training epochs')
# plt.legend(['train', 'val'], loc='best')

# ax6 = plt.subplot(336, sharex=ax4,sharey=ax4)

# plt.plot(history3.history['mae'])
# plt.plot(history3.history['val_mae'])
# # plt.title('model loss')
# # plt.ylabel('loss')
# # plt.xlabel('training epoch')
# plt.legend(['train', 'val'], loc='best')

# ax7 = plt.subplot(337)

# test_mae_loss=np.mean(np.abs(y_pred_test2[:,-1,0] - y_true_test), axis=1)
# sns.distplot(test_mae_loss, bins=30, kde=True)
# plt.ylabel('distribution')

# ax8 = plt.subplot(338,sharex=ax7,sharey=ax7)

# test_mae_loss=np.mean(np.abs(y_pred_test1[:,-1,0] - y_true_test), axis=1)
# sns.distplot(test_mae_loss, bins=30, kde=True)
# plt.xlabel('mae (test)')

# ax9 = plt.subplot(339,sharex=ax7,sharey=ax7)

# test_mae_loss=np.mean(np.abs(y_pred_test3 - y_true_test), axis=1)
# sns.distplot(test_mae_loss, bins=30, kde=True)

# plt.show()

# fig_name=os.path.join('figs',model_name+'_1_'+'summary'+'.pdf')
# fig.savefig(fig_name,dpi=600, bbox_inches="tight")